<a href="https://colab.research.google.com/github/rinalditoryanari/ModelSleepApneaEEG/blob/main/EEG_Sleep_Apnea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Library

In [ ]:
from google.colab import drive
import tensorflow as tf
import os
import numpy as np
import random
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
print(tf. __version__) 

2.12.0


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# TRY 1
Jadi ini tuh rencananya, data txt itu bakal dipecah 1 row di txt untuk 1 file text
Udah termasuk ngapus header nya

### eeg cap File

In [ ]:
sample_dir = "/content/drive/MyDrive/dataset/eegcap"
labels = ["normal", "sdb"]

##READ INDIVIDUAL

### CAP txt

#### window version

##### FFT

In [ ]:
from scipy.fft import fft

eeg_sample = []
eeg_labell = []
max_len = 0  # variable to store maximum length of signals
window_size = 879936
stride = 0.1 # 10% overlap

for label in labels:
    label_dir = os.path.join(sample_dir, label)
    for filename in os.listdir(label_dir):
        filepath = os.path.join(label_dir, filename)
        with open(filepath, 'r') as file:
            print(filename)
            # skip the first line as it contains column headers
            file.readline()

            # read the remaining lines and extract signal values
            data = []
            for i, line in enumerate(file):
                values = line.strip().split(";")[4] # extract signal value from the 5th column
                data.append(float(values))

            # Pad data if shorter than window size
            if len(data) < window_size:
                padding = np.zeros(window_size - len(data))
                data = np.concatenate([data, padding])

            # Split data into segments
            stride_size = int(window_size * stride)
            for i in range(0, len(data) - window_size + 1, stride_size):
                segment = data[i:i+window_size]

                # Apply FFT to the segment
                fft_segment = fft(segment)

                eeg_sample.append(segment)

                # if this is a new maximum length
                if len(segment) > max_len:
                    max_len = len(segment)

                if (label == 'normal'):
                    bi_label = 0
                elif (label == 'sdb'):
                    bi_label = 1

                eeg_labell.append(bi_label)

n1 c3p3.txt
sdb3 P4-O2 (2)
sdb3 P4-O2 (1)


####check

In [ ]:
eeg_sample = np.array(eeg_sample)
eeg_labell = np.array(eeg_labell)
print("EEG data shape:", eeg_sample.shape)
print("EEG label shape:", eeg_labell.shape)

EEG data shape: (352, 879936)
EEG label shape: (352,)


In [ ]:
unique_labels, label_counts = np.unique(eeg_labell, return_counts=True)
print("Label counts:", dict(zip(unique_labels, label_counts)))

Label counts: {0: 192, 1: 160}


In [ ]:
eeg_data = eeg_sample
eeg_labels = eeg_labell

### CNN

In [ ]:
eeg_data = np.array(eeg_data)

# check number of dimensions of eeg_data
print("EEG data dimensions:", eeg_data.ndim)

# swap second and third dimensions using swapaxes
# eeg_data = np.swapaxes(eeg_data, 1, 2)

# alternatively, you can reshape the array to match the desired axes
eeg_data = eeg_data.reshape(eeg_data.shape[0], eeg_data.shape[1], 1)
eeg_labels = eeg_labels.reshape(eeg_labels.shape[0], 1, 1)

# eeg_data = eeg_data.reshape(eeg_data.shape[0], eeg_data.shape[2], eeg_data.shape[1])

print("EEG data shape:", eeg_data.shape)
print("EEG label shape:", eeg_labels.shape)

EEG data dimensions: 2
EEG data shape: (352, 879936, 1)
EEG label shape: (352, 1, 1)


##SPLIT

###worked

In [ ]:
# Generate an array of indices for the samples
indices = np.arange(len(eeg_data))

# Shuffle the indices randomly
np.random.shuffle(indices)

# Use the shuffled indices to shuffle the data and labels arrays
eeg_data = eeg_data[indices]
eeg_labels = eeg_labels[indices]

In [ ]:
from collections import Counter


# Split the data into training, validation, and testing sets
num_samples = len(eeg_data)
num_train = int(num_samples * 0.7)
num_val = int(num_samples * 0.15)
num_test = num_samples - num_train - num_val

train_data = eeg_data[:num_train]
train_labels = eeg_labels[:num_train]

val_data = eeg_data[num_train:num_train+num_val]
val_labels = eeg_labels[num_train:num_train+num_val]

test_data = eeg_data[num_train+num_val:]
test_labels = eeg_labels[num_train+num_val:]

print("Number of training samples:", len(train_data))
print("Number of validation samples:", len(val_data))
print("Number of testing samples:", len(test_data))

Number of training samples: 246
Number of validation samples: 52
Number of testing samples: 54


In [ ]:
unique_labels, label_counts = np.unique(train_labels, return_counts=True)
print("training samples:", (dict(zip(unique_labels, label_counts))))

unique_labels, label_counts = np.unique(val_labels, return_counts=True)
print("validation samples:",(dict(zip(unique_labels, label_counts))))

unique_labels, label_counts = np.unique(test_labels, return_counts=True)
print("testing samples:", (dict(zip(unique_labels, label_counts))))

training samples: {0: 141, 1: 105}
validation samples: {0: 26, 1: 26}
testing samples: {0: 25, 1: 29}


# Modelling

## CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

cnn = Sequential()
cnn.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(879936, 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(16, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 879934, 32)        128       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 439967, 32)       0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 439965, 16)        1552      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 219982, 16)       0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3519712)           0         
                                                                 
 dense_2 (Dense)             (None, 16)               

# Execute Model

###CNN FFt

In [ ]:
cnn.fit(train_data, train_labels, batch_size=16, epochs=10, validation_data=(val_data, val_labels))

Epoch 1/10
16/16 [==============================] - 141s 9s/step - loss: 44.6973 - accuracy: 0.5407 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
16/16 [==============================] - 139s 9s/step - loss: 0.6814 - accuracy: 0.5732 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/10
16/16 [==============================] - 140s 9s/step - loss: 0.6877 - accuracy: 0.5732 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/10
16/16 [==============================] - 137s 9s/step - loss: 0.6864 - accuracy: 0.5732 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/10
16/16 [==============================] - 155s 10s/step - loss: 0.6837 - accuracy: 0.5732 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/10
16/16 [==============================] - 141s 9s/step - loss: 0.6828 - accuracy: 0.5732 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 7/10
16/16 [==============================] - 139s 9s/step - loss: 0.6824 - accuracy: 0.5732 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 8/10


In [ ]:
test_loss, test_accuracy = cnn.evaluate(test_data, test_labels)
print(f"Test loss: ", test_loss)
print(f"Test accuracy: ", test_accuracy)

2/2 [==============================] - 11s 4s/step - loss: 0.6987 - accuracy: 0.3889
Test loss:  0.698698878288269
Test accuracy:  0.3888888955116272
